# Observed Air Quality (PurpleAir)

This notebook retrieves readings from PurpleAir Sensors in Minneapolis and cleans the entries and saves the results as a csv file.

Documentation is available here: https://api.purpleair.com.
You can read this article for help getting started: https://community.purpleair.com/t/making-api-calls-with-the-purpleair-api/180.

From PurpleAir: 

"The data from individual sensors will update no less than every 30 seconds. As a courtesy, we ask that you limit the number of requests to no more than once every 1 to 10 minutes, assuming you are only using the API to obtain data from sensors. If retrieving data from multiple sensors at once, please send a single request rather than individual requests in succession.

The PurpleAir historical API is released as of July 18, 2022. For more information, view this post: https://community.purpleair.com/t/new-version-of-the-purpleair-api-on-july-18th/1251.

Please let us know if you have any questions or concerns, and have a great day!"

A paper on this process: https://doi.org/10.5194/amt-14-4617-2021 (Link for [Download](https://www.researchgate.net/publication/352663348_Development_and_application_of_a_United_States-wide_correction_for_PM25_data_collected_with_the_PurpleAir_sensor) )

Chat on which PM Estimate to use: https://community.purpleair.com/t/pm2-5-algorithms/3972/6

In [2]:
import os
import requests 
import datetime as dt
import pandas as pd
import arcpy
import numpy as np
import io

In [3]:
cwd = os.getcwd() # This is a global variable for where the notebook is (must change if running in arcpro)

# Make it workspace

arcpy.env.workspace = os.path.join(cwd, '..', '..', 'data', 'QAQC.gdb')

arcpy.env.overwriteOutput = True # Overwrite layers is okay

## Setting MPLS Bounds

In [4]:
#bound strings

mpls_8km = "mpls_8km"

bounds_strings = [f'nwlng=-93.43083707299996',
                  f'nwlat=45.12366876300007',
                  f'selng=-93.09225748799997',
                  f'selat=44.81791263300005']
bounds_string = '&'.join(bounds_strings)

print(bounds_string)

nwlng=-93.43083707299996&nwlat=45.12366876300007&selng=-93.09225748799997&selat=44.81791263300005


## Get Station IDs

In [5]:
# This function will be used to collect data for multiple public PurpleAir sensors.
def getSensorsData(query='', api_read_key=''):

    # my_url is assigned the URL we are going to send our request to.
    url = 'https://api.purpleair.com/v1/sensors?' + query
    
    print('Here is the full url for the API call:\n\n', url)

    # my_headers is assigned the context of our request we want to make. In this case
    # we will pass through our API read key using the variable created above.
    my_headers = {'X-API-Key':api_read_key}

    # This line creates and sends the request and then assigns its response to the
    # variable, r.
    response = requests.get(url, headers=my_headers)

    # We then return the response we received.
    return response

In [6]:
# This is my personal API key... Please use responsibly!
# 51592903-B445-11ED-B6F4-42010A800007

api = input('Please enter your Purple Air api key')

Please enter your Purple Air api key 51592903-B445-11ED-B6F4-42010A800007


In [7]:
# Designating and formatting the fields to request

fields = ['location_type']

fields_string = 'fields=' + '%2C'.join(fields)

print(fields_string)

fields=location_type


In [8]:
# Put it all together

query_string = '&'.join([fields_string, bounds_string])

print(query_string)

fields=location_type&nwlng=-93.43083707299996&nwlat=45.12366876300007&selng=-93.09225748799997&selat=44.81791263300005


In [9]:
# Make the request

response = getSensorsData(query_string, api)

Here is the full url for the API call:

 https://api.purpleair.com/v1/sensors?fields=location_type&nwlng=-93.43083707299996&nwlat=45.12366876300007&selng=-93.09225748799997&selat=44.81791263300005


In [10]:
# Get response into Pandas DataFrame

response_dict = response.json() # Read response as a json (dictionary)

col_names = response_dict['fields']
data = np.array(response_dict['data'])

df = pd.DataFrame(data, columns = col_names)

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 87 entries, 0 to 86
Data columns (total 2 columns):
 #   Column         Non-Null Count  Dtype
---  ------         --------------  -----
 0   sensor_index   87 non-null     int32
 1   location_type  87 non-null     int32
dtypes: int32(2)
memory usage: 824.0 bytes


In [11]:
df.head()

,sensor_index,location_type
0,3088,0
1,5582,0
2,137876,1
3,11134,0
4,142718,0


In [12]:
# Only want outside sensors

outside_sensors = df[df['location_type']==0] # 0 = outside

len(outside_sensors)

79

In [14]:
response.json()

{'api_version': 'V1.0.11-0.0.42',
 'time_stamp': 1680562985,
 'data_time_stamp': 1680562925,
 'max_age': 604800,
 'firmware_default_version': '7.02',
 'fields': ['sensor_index', 'location_type'],
 'location_types': ['outside', 'inside'],
 'data': [[3088, 0],
  [5582, 0],
  [137876, 1],
  [11134, 0],
  [142718, 0],
  [142720, 0],
  [142726, 0],
  [142730, 0],
  [142728, 0],
  [142734, 0],
  [142732, 0],
  [142736, 0],
  [142744, 0],
  [142750, 0],
  [142748, 0],
  [142752, 0],
  [142756, 0],
  [142774, 0],
  [142772, 0],
  [142852, 0],
  [143214, 0],
  [143216, 0],
  [143222, 0],
  [143226, 0],
  [143224, 0],
  [143238, 0],
  [143242, 0],
  [143240, 0],
  [143246, 0],
  [143636, 0],
  [143648, 0],
  [143656, 0],
  [143666, 0],
  [143668, 0],
  [143916, 0],
  [145202, 0],
  [145204, 0],
  [145234, 0],
  [145242, 0],
  [145250, 0],
  [145454, 0],
  [145470, 0],
  [145498, 0],
  [145506, 0],
  [145604, 0],
  [145610, 0],
  [145616, 0],
  [147749, 1],
  [17189, 1],
  [21179, 0],
  [154751, 

In [13]:
#drop the location_type now that we have filtered for outdoor sensors only
df_historic = outside_sensors.drop('location_type', axis=1)

## Pulling Historic Sensor CSVs

### Setting time period

In [18]:
#pulling from 9/1/22 - 4/2/23 

# Start time

end_datetime = dt.datetime(2023,4,2) # April 2, 2023
end_timestamp = int(dt.datetime.timestamp(end_datetime))

# End time

start_datetime = dt.datetime(2022,9,1) # September 1, 2022
start_timestamp = int(dt.datetime.timestamp(start_datetime))

# Sensors

sensor_ids = df_historic.sensor_index.apply(lambda x: int(x))
sensor_ids

0       3088
1       5582
3      11134
4     142718
5     142720
       ...  
82    108366
83    113486
84    116413
85    126487
86    128195
Name: sensor_index, Length: 79, dtype: int64

### Creating the Query for the API

### Calling a SINGLE sensor

In [32]:
#DO NOT EDIT - KEEP AS IS FOR NOW
# Sensor id

sensor_id = sensor_ids[0]

# Timestamp String

time_string = 'start_timestamp=' + str(start_timestamp) + '&end_timestamp=' + str(end_timestamp)

# Average string (in minutes) 1440 is 1 day average

avg_string = 'average=1440'

# Environmental fields

env_fields = ['humidity', 'temperature', 'pressure', 'pm2.5_cf_1']

env_fields_string = 'fields=' + '%2C%20'.join(env_fields)

# Base URL

base_url = f'https://api.purpleair.com/v1/sensors/{sensor_id}/history/csv?'

# Put it all together

query_url = base_url + '&'.join([time_string, avg_string, env_fields_string])

my_headers = {'X-API-Key':api}

# This line creates and sends the request and then assigns its response to the
# variable, r.
r = requests.get(query_url, headers=my_headers)

# Read response as CSV data
csv_data = r.content.decode('utf-8')

# Parse CSV data into pandas DataFrame
df_historic = pd.read_csv(io.StringIO(csv_data), header=None)
df_historic = df_historic.iloc[1:]  # exclude the header row
df_historic.columns = ['timestamp'] + ['sensor_index'] + env_fields
df_historic

,humidity,temperature,pressure,pm2.5_cf_1,timestamp,pm2.5_cf_1
0,1674864000,3088,40.201,17.581,991.743,0.6330
1,1671753600,3088,42.605,0.743,987.762,0.0785
2,1668729600,3088,48.795,29.080,990.701,54.4190
3,1672531200,3088,53.942,42.170,980.359,8.8820
4,1676764800,3088,50.660,40.259,978.572,7.8060
...,...,...,...,...,...,...
208,1677628800,3088,52.806,44.600,977.058,3.9945
209,1679356800,3088,41.454,42.326,987.113,2.3800
210,1676592000,3088,38.144,22.964,995.503,0.5600
211,1679529600,3088,41.673,41.035,988.664,3.2915


### Creating a 'for' loop to parse through ALL sensor_ids

### Setting Parameters for API

In [35]:
sensor_id = sensor_ids[0]

#Creating an empty dataframe to hold all sensor data
df_all_sensors = pd.DataFrame()
   
# Timestamp String

time_string = 'start_timestamp=' + str(start_timestamp) + '&end_timestamp=' + str(end_timestamp)

# Average string (in minutes) 1440 is 1 day average

avg_string = 'average=1440'

# Environmental fields

env_fields = ['humidity', 'temperature', 'pressure', 'pm2.5_cf_1']

env_fields_string = 'fields=' + '%2C%20'.join(env_fields)

### The For Loop

In [36]:
for sensor_id in sensor_ids:
    # Base URL
    base_url = f'https://api.purpleair.com/v1/sensors/{sensor_id}/history/csv?'

    # Put it all together
    query_url = base_url + '&'.join([time_string, avg_string, env_fields_string])

    my_headers = {'X-API-Key': api}
    r = requests.get(query_url, headers=my_headers)

    if r.status_code == 200:
        # Read response as CSV data
        csv_data = r.content.decode('utf-8')

        # Parse CSV data into pandas DataFrame
        df_individual_sensor = pd.read_csv(io.StringIO(csv_data), header=None)
        df_individual_sensor = df_individual_sensor.iloc[1:]  # exclude the header row
        df_individual_sensor.columns = ['timestamp'] + ['sensor_index'] + env_fields

        # Concatenate the individual sensor data with the main DataFrame
        df_all_sensors = pd.concat([df_all_sensors, df_individual_sensor], ignore_index=True)
    else:
        print(f"Error fetching data for sensor {sensor_id}: {r.status_code}")

# Display the concatenated DataFrame for all sensors
df_all_sensors

,timestamp,sensor_index,humidity,temperature,pressure,pm2.5_cf_1
0,1674864000,3088,40.201,17.581,991.743,0.633
1,1671753600,3088,42.605,0.743,987.762,0.0785
2,1668729600,3088,48.795,29.08,990.701,54.419000000000004
3,1672531200,3088,53.942,42.17,980.359,8.882
4,1676764800,3088,50.66,40.259,978.572,7.806000000000001
...,...,...,...,...,...,...
12579,1668988800,128195,49.34,33.981,985.172,6.12
12580,1678492800,128195,57.387,37.036,985.28,0.6775
12581,1673827200,128195,64.146,41.889,969.533,12.6325
12582,1673913600,128195,66.95,40.677,970.189,4.1875


## Cleaning Historic Data for Analysis

In [56]:
#rename pm2.5 column to pm2_5 for SQL
df_historic = df_all_sensors.rename(columns={'pm2.5_cf_1' : 'pm2_5'})

In [57]:
#changing UNIX date to pd date
df_historic['timestamp'] = pd.to_datetime(df_historic['timestamp'], unit='s')
df_historic

,timestamp,sensor_index,humidity,temperature,pressure,pm2_5
0,2023-01-28,3088,40.201,17.581,991.743,0.633
1,2022-12-23,3088,42.605,0.743,987.762,0.0785
2,2022-11-18,3088,48.795,29.08,990.701,54.419000000000004
3,2023-01-01,3088,53.942,42.17,980.359,8.882
4,2023-02-19,3088,50.66,40.259,978.572,7.806000000000001
...,...,...,...,...,...,...
12579,2022-11-21,128195,49.34,33.981,985.172,6.12
12580,2023-03-11,128195,57.387,37.036,985.28,0.6775
12581,2023-01-16,128195,64.146,41.889,969.533,12.6325
12582,2023-01-17,128195,66.95,40.677,970.189,4.1875


## QAQC

In [58]:
#create a blank dataframe to hold the errors

purpleair_historic_errors = pd.DataFrame(columns = ['humidity_error', 'temperature_error', 'pressure_error', 'pm2_5_error'])
purpleair_historic_errors['sensor_index'] = df_historic['sensor_index']
purpleair_historic_errors['timestamp'] = df_historic['timestamp']

### Humidity Check

In [42]:
#ranges pulled from https://www.currentresults.com/Weather/Minnesota/humidity-annual.php
#range is actually 40-90 but I was getting tons of errors so I reduced it to 10-90

import pandas as pd

def check_range(value):
    if pd.isna(value):
        return 'no value given'
    elif value >= 10 and value <= 90:
        pass
    else:
        return 'out of range (10%-90%)'

# Assuming 'df_historic' is your original dataframe

# Convert 'humidity' column to float
df_historic['humidity'] = pd.to_numeric(df_historic['humidity'], errors='coerce')

# Apply 'check_range' function to 'humidity' column and store the result in a new column 'humidity_error'
df_historic['humidity_error'] = df_historic['humidity'].apply(check_range)

print(df_historic)

       timestamp sensor_index  humidity temperature pressure  \
0     2023-01-28         3088    40.201      17.581  991.743   
1     2022-12-23         3088    42.605       0.743  987.762   
2     2022-11-18         3088    48.795       29.08  990.701   
3     2023-01-01         3088    53.942       42.17  980.359   
4     2023-02-19         3088    50.660      40.259  978.572   
...          ...          ...       ...         ...      ...   
12579 2022-11-21       128195    49.340      33.981  985.172   
12580 2023-03-11       128195    57.387      37.036   985.28   
12581 2023-01-16       128195    64.146      41.889  969.533   
12582 2023-01-17       128195    66.950      40.677  970.189   
12583 2022-09-04       128195    45.508      71.343  990.908   

                    pm2_5 humidity_error  
0                   0.633           None  
1                  0.0785           None  
2      54.419000000000004           None  
3                   8.882           None  
4       7.806000

### Temperature Check

In [44]:
#winter -4 - 28
#spring 22 - 57
#summer 48 - 81
#fall 29 - 59
#ref from https://www.dnr.state.mn.us/climate/summaries_and_publications/normalsportal.html


def check_range(value):
    if pd.isna(value):
        return 'no value given'
    elif float(value) >= -20 and float(value) <= 100:
        pass
    else:
        return 'out of range (-20-100F)'

'''
#if we can get time stamp we should use this with a date check too
#this is not correct - we can do seasonal if we can relate it to date range
def check_range(value):
    if value is None:
        return -1
    if value >= -20 and value <=35:
        return 'winter (-20-35F)'
    if value >10 and value <=70:
        return 'spring (10-70F)'
    if value >30 and value <=100:
        return 'summer (30-100F)'
    if value >15 and value <=70:
        return 'fall (15-70F)'
    else:
        return 'out of range'
'''

# Assuming 'df_historic' is your original dataframe

# Convert 'temperature' column to float
df_historic['temperature'] = pd.to_numeric(df_historic['temperature'], errors='coerce')

# Apply 'check_range' function to 'temperature' column and store the result in a new column 'temperature_error'
df_historic['temperature_error'] = df_historic['temperature'].apply(check_range)

print(df_historic)


       timestamp sensor_index  humidity  temperature pressure  \
0     2023-01-28         3088    40.201       17.581  991.743   
1     2022-12-23         3088    42.605        0.743  987.762   
2     2022-11-18         3088    48.795       29.080  990.701   
3     2023-01-01         3088    53.942       42.170  980.359   
4     2023-02-19         3088    50.660       40.259  978.572   
...          ...          ...       ...          ...      ...   
12579 2022-11-21       128195    49.340       33.981  985.172   
12580 2023-03-11       128195    57.387       37.036   985.28   
12581 2023-01-16       128195    64.146       41.889  969.533   
12582 2023-01-17       128195    66.950       40.677  970.189   
12583 2022-09-04       128195    45.508       71.343  990.908   

                    pm2_5 humidity_error temperature_error  
0                   0.633           None              None  
1                  0.0785           None              None  
2      54.419000000000004           

### Pressure Check

In [46]:
# range is 25 - 35 Hg according to https://barometricpressure.app/minneapolis
# PurpleAir uses Millibars so I used https://www.weather.gov/epz/wxcalc_pressureconvert to convert
# rage is 846.6 - 1185.24

import pandas as pd

def check_range(value):
    if pd.isna(value):
        return 'no value given'
    elif float(value) >= 830 and float(value) <= 1200:
        pass
    else:
        return 'out of range (830 - 1200 Millibars)'

# Assuming 'df_historic' is your original dataframe

# Convert 'pressure' column to float
df_historic['pressure'] = pd.to_numeric(df_historic['pressure'], errors='coerce')

# Apply 'check_range' function to 'pressure' column and store the result in a new column 'pressure_error'
df_historic['pressure_error'] = df_historic['pressure'].apply(check_range)

print(df_historic)


       timestamp sensor_index  humidity  temperature  pressure  \
0     2023-01-28         3088    40.201       17.581   991.743   
1     2022-12-23         3088    42.605        0.743   987.762   
2     2022-11-18         3088    48.795       29.080   990.701   
3     2023-01-01         3088    53.942       42.170   980.359   
4     2023-02-19         3088    50.660       40.259   978.572   
...          ...          ...       ...          ...       ...   
12579 2022-11-21       128195    49.340       33.981   985.172   
12580 2023-03-11       128195    57.387       37.036   985.280   
12581 2023-01-16       128195    64.146       41.889   969.533   
12582 2023-01-17       128195    66.950       40.677   970.189   
12583 2022-09-04       128195    45.508       71.343   990.908   

                    pm2_5 humidity_error temperature_error pressure_error  
0                   0.633           None              None           None  
1                  0.0785           None              N

### PM Check

In [59]:
#Average reading in MPLS is 30 ug/m3 per https://www.epa.gov/air-trends/air-quality-cities-and-counties

def check_range(value):
    if value is None:
        return 'no value given'
    try:
        value = float(value)  # Convert value to float
    except ValueError:
        return 'invalid value'
    
    if 0.1 < value < 1000:  # Use numerical comparison on float value
        pass
    else:
        return 'above 1000'


#    if value == 0:
 #       return '0'
#    if value >0.1 and value <=10:
#        return 'PM2.5 0.1-10'
#    if value >10 and value <=20:
#        return 'PM2.5 10-20'
#    if value >20 and value <=30:
#        return 'PM2.5 20-30'
#    if value >30 and value <=40:
#        return 'PM2.5 30-40'
#    if value >40 and value <=50:
#        return 'PM2.5 40-50'
#    if value >50 and value <=60:
#        return 'PM2.5 50-60'
#    if value >60 and value <=70:
#        return 'PM2.5 60-70'

    
purpleair_historic_errors['pm2_5_error'] = df_historic['pm2_5'].apply(check_range)

print(purpleair_historic_errors)

      humidity_error temperature_error pressure_error pm2_5_error  \
0                NaN               NaN            NaN        None   
1                NaN               NaN            NaN  above 1000   
2                NaN               NaN            NaN        None   
3                NaN               NaN            NaN        None   
4                NaN               NaN            NaN        None   
...              ...               ...            ...         ...   
12579            NaN               NaN            NaN         NaN   
12580            NaN               NaN            NaN         NaN   
12581            NaN               NaN            NaN         NaN   
12582            NaN               NaN            NaN         NaN   
12583            NaN               NaN            NaN         NaN   

      sensor_index  timestamp  
0             3088 2023-01-28  
1             3088 2022-12-23  
2             3088 2022-11-18  
3             3088 2023-01-01  
4          

In [60]:
# Removing rows from the error table that don't have any errors

purpleair_historic_errors = purpleair_historic_errors.dropna(subset=purpleair_historic_errors.columns.difference(['sensor_index', 'timestamp']), how='all')
purpleair_historic_errors

,humidity_error,temperature_error,pressure_error,pm2_5_error,sensor_index,timestamp
1,NaN,NaN,NaN,above 1000,3088,2022-12-23
58,NaN,NaN,NaN,above 1000,3088,2022-10-18
137,NaN,NaN,NaN,above 1000,3088,2022-11-12
188,NaN,NaN,NaN,above 1000,3088,2022-10-17
193,NaN,NaN,NaN,above 1000,3088,2022-12-24
...,...,...,...,...,...,...
12529,NaN,NaN,NaN,above 1000,128195,2023-03-19
12535,NaN,NaN,NaN,above 1000,128195,2022-10-15
12546,NaN,NaN,NaN,above 1000,128195,2022-09-29
12552,NaN,NaN,NaN,above 1000,128195,2023-02-03


## Connecting to the Server

In [61]:
import psycopg2
from psycopg2 import sql

In [69]:
connection = psycopg2.connect(host = '34.27.240.213',
                              database = 'lab0',
                              user = 'postgres',
                              password = 'Homework13',
                              port = '5432')
connection.closed

0

## Insert Data into SQL Table

In [70]:
# connect to the cursor
cur = connection.cursor()

# iterate over the dataframe and insert each row into the database using a SQL INSERT statement
for index, row in df_historic.iterrows():
    cur.execute('''
    INSERT INTO PURPLEAIR_HISTORIC (sensor_index, timestamp, humidity, temperature, pressure, pm2_5) 
    VALUES (%s, %s, %s, %s, %s, %s) 
    ''', (row['sensor_index'], row['timestamp'], row['humidity'], row['temperature'], row['pressure'], row['pm2_5']))
    connection.commit()

for i, r in purpleair_historic_errors.iterrows():
    cur.execute('''
    INSERT INTO PURPLEAIR_HISTORIC_ERRORS (sensor_index, timestamp, humidity_error, temperature_error, pressure_error, pm2_5_error) 
    VALUES (%s, %s, %s, %s, %s, %s) 
    ''', (r['sensor_index'], r['timestamp'], r['humidity_error'], r['temperature_error'], r['pressure_error'], r['pm2_5_error']))
    connection.commit()

# commit the changes to the database and close the cursor and connection
cur.close()
connection.close()